In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
# import turicreate as tc  # Only Mac/Linux
from scipy.sparse.linalg import svds

In [2]:
ratings = pd.read_csv('ratings.csv')
books = pd.read_csv('books_enriched.csv', index_col = 'book_id')

In [8]:
ratings.shape

(5976479, 3)

In [5]:
# Add books to the df
new_ratings = pd.read_csv('new_ratings.csv')
new_ratings

,user_id,book_id,rating
0,53425,34,4
1,53425,99,3
2,53425,2,3
3,53425,40,5
4,53425,29,4
5,53425,257,2
6,53425,365,3


In [9]:
ratings = pd.concat([ratings, new_ratings])
ratings.shape

(5976486, 3)

In [10]:
# ratings = ratings[ratings['book_id'] <= 1000]

In [11]:
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [12]:
max(ratings['user_id'])

53425

In [5]:
ratings.duplicated(['user_id','book_id','rating']).sum()
# No duplicate ratings

0

In [6]:
# Ratings per users
ratings.groupby(by = "user_id")["rating"].count().sort_values(ascending = False)

user_id
45554    173
13879    172
46139    172
13925    171
8440     167
        ... 
32925      1
52083      1
39283      1
10620      1
23378      1
Name: rating, Length: 53417, dtype: int64

In [7]:
# Ratings per book
ratings.groupby(by = "book_id")["rating"].count().sort_values(ascending = False)

book_id
1      22806
2      21850
4      19088
3      16931
5      16604
       ...  
524      386
954      372
845      332
580      310
990      278
Name: rating, Length: 1000, dtype: int64

In [8]:
books.loc[ratings.groupby(by = "book_id")["rating"].count().sort_values(ascending = False).index[:20], 'original_title']

book_id
1                              The Hunger Games
2      Harry Potter and the Philosopher's Stone
4                         To Kill a Mockingbird
3                                      Twilight
5                              The Great Gatsby
17                                Catching Fire
20                                   Mockingjay
18     Harry Potter and the Prisoner of Azkaban
23      Harry Potter and the Chamber of Secrets
7            The Hobbit or There and Back Again
24          Harry Potter and the Goblet of Fire
25         Harry Potter and the Deathly Hallows
21    Harry Potter and the Order of the Phoenix
27       Harry Potter and the Half-Blood Prince
13                         Nineteen Eighty-Four
8                        The Catcher in the Rye
16                        Män som hatar kvinnor
14                   Animal Farm: A Fairy Story
28                           Lord of the Flies 
9                              Angels & Demons 
Name: original_title, dtype: obj

In [9]:
# Get sparse matrix - user_id in rows, book_id in cols, 0 if rating not present, else rating value
def get_user_item_sparse_matrix(df):
    sparse_data = sparse.csr_matrix((df.rating, (df.user_id, df.book_id)))
    return sparse_data

In [10]:
sparse_ratings = get_user_item_sparse_matrix(ratings)

In [11]:
# Global average rating
sparse_ratings.sum()/sparse_ratings.count_nonzero()

3.9344739332834204

In [12]:
def get_average_rating(sparse_matrix, is_user):
    ax = 1 if is_user else 0
    sum_of_ratings = sparse_matrix.sum(axis = ax).A1  
    no_of_ratings = (sparse_matrix != 0).sum(axis = ax).A1 
    rows, cols = sparse_matrix.shape
    average_ratings = {i: sum_of_ratings[i]/no_of_ratings[i] for i in range(rows if is_user else cols) if no_of_ratings[i] != 0}
    return average_ratings

In [13]:
# Average Rating User
average_rating_user = get_average_rating(sparse_ratings, True)

In [14]:
# Average Rating Book
avg_rating_book = get_average_rating(sparse_ratings, False)

In [15]:
# # of users not present in data (no need to run this)
total_users = len(np.unique(ratings["user_id"]))
train_users = len(average_rating_user)
uncommonUsers = total_users - train_users
                  
print("Total no. of Users = {}".format(total_users))
print("No. of Users in train data= {}".format(train_users))
print("No. of Users not present in train data = {}({}%)".format(uncommonUsers, np.round((uncommonUsers/total_users)*100), 2))

Total no. of Users = 53417
No. of Users in train data= 53417
No. of Users not present in train data = 0(0.0%)


In [16]:
# # of books not present in data (no need to run this)
total_books = len(np.unique(ratings["book_id"]))
train_books = len(avg_rating_book)
uncommonBooks = total_books - train_books
                  
print("Total no. of Books = {}".format(total_books))
print("No. of Books in train data= {}".format(train_books))
print("No. of Books not present in train data = {}({}%)".format(uncommonBooks, np.round((uncommonBooks/total_books)*100), 2))

Total no. of Books = 1000
No. of Books in train data= 1000
No. of Books not present in train data = 0(0.0%)


In [17]:
sparse_ratings.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 5, ..., 0, 0, 0],
        ...,
        [0, 4, 5, ..., 0, 0, 0],
        [0, 4, 5, ..., 0, 0, 0],
        [0, 4, 5, ..., 0, 0, 0]], dtype=int64)

In [25]:
books_sim_corr = np.corrcoef(sparse_ratings.todense()[:,1:], rowvar = False)

In [26]:
p = 0
print('Most similar books to :',books.loc[p+1, 'original_title'],'based on correlation are:')
for i in np.argsort(books_sim_corr[p])[-2:-7:-1]:
    print(books.loc[i+1, 'original_title'])

Most similar books to : The Hunger Games based on correlation are:
Catching Fire
Mockingjay
Twilight
Divergent
Harry Potter and the Philosopher's Stone


In [27]:
p = 12
print('Most similar books to :',books.loc[p+1, 'original_title'],'based on correlation are:')
for i in np.argsort(books_sim_corr[p])[-2:-7:-1]:
    print(books.loc[i+1, 'original_title'])

Most similar books to : Nineteen Eighty-Four based on correlation are:
Animal Farm: A Fairy Story
Brave New World
Fahrenheit 451
Lord of the Flies 
The Catcher in the Rye


In [22]:
books_sim_cos = cosine_similarity(sparse_ratings.todense().T)

C:\Users\vishw\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [23]:
p = 0
print('Most similar books to :',books.loc[p+1, 'original_title'],'based on cosine similarity are:')
for i in np.argsort(books_sim_cos[p+1])[-2:-7:-1]:
    print(books.loc[i+1, 'original_title'])

Most similar books to : The Hunger Games based on cosine similarity are:
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Order of the Phoenix
Twilight
To Kill a Mockingbird
Nineteen Eighty-Four


In [24]:
p = 12
print('Most similar books to :',books.loc[p+1, 'original_title'],'based on cosine similarity are:')
for i in np.argsort(books_sim_cos[p+1])[-2:-7:-1]:
    print(books.loc[i+1, 'original_title'])

Most similar books to : Nineteen Eighty-Four based on cosine similarity are:
Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
Breaking Dawn
Angels & Demons 
An Excellent conceited Tragedie of Romeo and Juliet
The Fault in Our Stars


In [13]:
ratings_df = ratings.pivot(index = 'user_id', columns ='book_id', values = 'rating').fillna(0)
ratings_df.head()

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,5.0,0.0,4.0,4.0,0.0,4.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
type(ratings_df)

pandas.core.frame.DataFrame

In [15]:
ratings_matrix = ratings_df.values
user_ratings_mean = np.mean(ratings_matrix, axis = 1)
ratings_demeaned = ratings_matrix - user_ratings_mean.reshape(-1, 1)
ratings_demeaned

array([[-4.2000e-02, -4.2000e-02, -4.2000e-02, ..., -4.2000e-02,
        -4.2000e-02, -4.2000e-02],
       [-2.8700e-02,  4.9713e+00, -2.8700e-02, ..., -2.8700e-02,
        -2.8700e-02, -2.8700e-02],
       [-1.5800e-02, -1.5800e-02, -1.5800e-02, ..., -1.5800e-02,
        -1.5800e-02, -1.5800e-02],
       ...,
       [ 3.9657e+00,  4.9657e+00, -3.4300e-02, ..., -3.4300e-02,
        -3.4300e-02, -3.4300e-02],
       [ 3.9414e+00,  4.9414e+00,  3.9414e+00, ..., -5.8600e-02,
        -5.8600e-02, -5.8600e-02],
       [-2.4000e-03,  2.9976e+00, -2.4000e-03, ..., -2.4000e-03,
        -2.4000e-03, -2.4000e-03]])

In [16]:
U, sigma, Vt = svds(ratings_demeaned, k = 50)
sigma = np.diag(sigma)

In [17]:
books = pd.read_csv('books_enriched.csv')

In [18]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = ratings_df.columns)
preds_df

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
0,0.566212,0.200623,0.239290,2.981033,2.009097,0.724402,0.246358,1.398149,0.363885,3.986424,...,0.054892,-0.000481,-0.007057,-0.004621,0.010961,0.014348,0.004063,0.000254,-0.003378,-0.007225
1,0.078133,4.485574,0.169710,2.444561,1.961953,0.644629,0.635967,1.709596,1.040178,2.794757,...,0.000462,0.003780,0.008717,-0.012557,-0.002044,0.010541,0.036487,0.003755,0.036672,0.038279
2,-0.031919,0.078681,-0.135188,1.637185,0.927012,-0.180757,0.209129,0.956740,-0.094447,0.397979,...,0.014760,0.008303,0.008942,0.009338,0.013517,0.003029,0.010829,0.005794,0.013871,0.011592
3,0.018909,4.534398,-0.041078,5.131711,3.153053,-0.854882,2.633743,3.787978,1.953858,2.521627,...,0.007306,-0.043097,-0.023740,0.018296,0.012278,0.010639,-0.008373,-0.000971,-0.006310,0.000703
4,-0.239790,-0.111398,-0.088481,0.123275,0.199841,1.509025,-0.066550,0.099760,0.102092,0.108077,...,0.024209,0.034106,0.025075,0.014417,0.017412,0.027896,0.021651,0.025852,0.032679,0.017620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,4.899336,3.707652,0.672806,5.392508,3.029217,-0.194066,3.524484,2.127661,5.233740,0.179993,...,-0.001557,0.028662,0.025614,0.008711,0.051199,-0.018282,0.038922,0.065362,-0.005856,0.058208
53421,4.483485,5.049251,0.562749,0.947642,0.601602,2.067604,4.065408,0.076584,0.300662,1.795913,...,-0.016192,-0.073189,-0.001047,0.039827,-0.007708,-0.023739,-0.013875,-0.011012,0.001145,-0.008836
53422,2.167565,5.105633,0.430062,3.755224,1.864624,0.938651,1.801304,1.573648,-0.062918,0.662248,...,0.001564,-0.003444,0.013889,0.031564,0.003844,0.000589,0.017221,0.011249,-0.002498,0.016656
53423,4.378529,4.307632,3.269371,4.105135,1.600688,0.857258,4.579557,0.560393,-0.424756,3.507097,...,-0.022757,0.064549,0.005317,0.113040,0.004812,0.000856,-0.015253,0.019990,-0.010542,-0.022382


In [19]:
def recommend_books(predictions_df, userID, books_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the book information.
    user_data = original_ratings_df[original_ratings_df['user_id'] == (userID)]
    user_full = (user_data.merge(books_df, how = 'left', left_on = 'book_id', right_on = 'book_id').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} books. The top 5 are:'.format(userID, user_full.shape[0]))
    print (user_full['title'].head(5))
    print ('Recommending the highest {0} predicted ratings books not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating book that the user hasn't seen yet.
    recommendations = (books_df[~books_df['book_id'].isin(user_full['book_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'book_id',
               right_on = 'book_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

#    return user_full, recommendations
    return recommendations[['book_id','title']]


In [20]:
recommend_books(preds_df, 53425, books, ratings, 3)

User 53425 has already rated 7 books. The top 5 are:
3                                      Eat, Pray, Love
0              Fifty Shades of Grey (Fifty Shades, #1)
4                                     Romeo and Juliet
1               Fifty Shades Darker (Fifty Shades, #2)
2    Harry Potter and the Sorcerer's Stone (Harry P...
Name: title, dtype: object
Recommending the highest 3 predicted ratings books not already rated.


,book_id,title
1,3,"Twilight (Twilight, #1)"
0,1,"The Hunger Games (The Hunger Games, #1)"
11,15,The Diary of a Young Girl


In [132]:
recommend_books(preds_df, 110, books, ratings, 3)

User 110 has already rated 38 books. The top 5 are:
20                                    The Metamorphosis
32                         The Adventures of Tom Sawyer
3     Harry Potter and the Deathly Hallows (Harry Po...
31                   The Adventures of Huckleberry Finn
22                                            The Trial
Name: title, dtype: object
Recommending the highest 3 predicted ratings books not already rated.


,book_id,title
16,23,Harry Potter and the Chamber of Secrets (Harry...
41,55,Brave New World
19,29,Romeo and Juliet


In [136]:
recommend_books(preds_df, 980, books, ratings, 3)

User 980 has already rated 73 books. The top 5 are:
0                       To Kill a Mockingbird
52    The Hunger Games (The Hunger Games, #1)
31                   A Thousand Splendid Suns
39                            The Thorn Birds
42                          The Kitchen House
Name: title, dtype: object
Recommending the highest 3 predicted ratings books not already rated.


,book_id,title
17,33,Memoirs of a Geisha
10,20,"Mockingjay (The Hunger Games, #3)"
32,57,The Secret Life of Bees
